<a href="https://colab.research.google.com/github/szkics/GerLangReconstruct/blob/main/machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [22]:
input_word_list = []
target_word_list = []
with open("/content/gdrive/MyDrive/gerlangreconstruct/german_english_dictionary.csv") as f:
    for line in f:
      splitted_line = line.split(",")
      input_word = splitted_line[0].lower()
      input_word_to_append = ''.join(e for e in input_word if e.isalnum() and not e.isdigit())
      input_word_list.append(input_word_to_append)
      target_word = splitted_line[1].lower()
      target_word_to_append = ''.join(e for e in target_word if e.isalnum() and not e.isdigit())
      target_word_list.append(target_word_to_append)
len(input_word_list)

257663

In [23]:
len(target_word_list)
input_word_list = input_word_list[:1000]
target_word_list = target_word_list[:1000]
len(target_word_list)

1000

In [25]:
import collections

import helper
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [26]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8274619063317402697
]


In [27]:
# Load English data
english_sentences = input_word_list #helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = target_word_list#helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


In [28]:
for sample_i in range(5):
    print('German sample {}:  {}'.format(sample_i + 1, german_sentences[sample_i]))
    print('French sample {}:  {}\n'.format(sample_i + 1, english_sentences[sample_i]))

English sample 1:  aale
French sample 1:  eels

English sample 2:  aalfell
French sample 2:  eelskin

English sample 3:  aalfischer
French sample 3:  eeler

English sample 4:  aalfänger
French sample 4:  eeler

English sample 5:  aalfänger
French sample 5:  sniggler



In [32]:
def count_letters(input_word_list):
  input_letter_counter = {}
  alphabet = {}
  unique_index = 0
  for word in input_word_list:
    for letter in list(word):
      if letter in input_letter_counter:
        input_letter_counter[letter] = input_letter_counter[letter] + 1
      else:
        input_letter_counter[letter] = 1
      if letter not in alphabet:
        alphabet[letter] = unique_index
        unique_index = unique_index + 1
  return input_letter_counter, alphabet

input_letter_counter, input_alphabet = count_letters(german_sentences)
target_letter_counter, output_alphabet = count_letters(english_sentences)

In [33]:
input_letter_counter

{'a': 1583,
 'b': 1305,
 'c': 244,
 'd': 376,
 'e': 1260,
 'f': 313,
 'g': 622,
 'h': 321,
 'i': 420,
 'j': 11,
 'k': 180,
 'l': 510,
 'm': 121,
 'n': 720,
 'o': 149,
 'p': 45,
 'r': 596,
 's': 354,
 't': 513,
 'u': 417,
 'v': 28,
 'w': 21,
 'x': 2,
 'y': 3,
 'z': 51,
 'ß': 10,
 'ä': 85,
 'é': 1,
 'ö': 10,
 'ü': 64}

In [39]:
input_alphabet

{'a': 0,
 'b': 22,
 'c': 6,
 'd': 21,
 'e': 2,
 'f': 3,
 'g': 11,
 'h': 7,
 'i': 4,
 'j': 16,
 'k': 24,
 'l': 1,
 'm': 13,
 'n': 10,
 'o': 17,
 'p': 15,
 'r': 8,
 's': 5,
 't': 14,
 'u': 19,
 'v': 18,
 'w': 20,
 'x': 27,
 'y': 25,
 'z': 23,
 'ß': 28,
 'ä': 9,
 'é': 29,
 'ö': 12,
 'ü': 26}

In [150]:
def tokenize(word, alphabet):
  vectorized = []
  for letter in list(word):
    vectorized.append(alphabet[letter])
  return vectorized

tokenized_word = tokenize("fahrrad", input_alphabet)

In [72]:
tokenized_word

[3, 0, 7, 8, 8, 0, 21]

In [75]:
def pad(x):
  y = x
  i = 0
  number_of_zeros = 42-len(x)
  while (i < number_of_zeros):
      y.append(0)
      i = i + 1
  return y
padded_word = pad(tokenized_word)

In [172]:
padded_word

[3,
 0,
 7,
 8,
 8,
 0,
 21,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [173]:
def preprocess(x, y, input_alphabet, target_alphabet):

    preprocess_x = []
    preprocess_y = []

    for word in x:
      preprocess_word = tokenize(word, input_alphabet)
      preprocess_x.append(preprocess_word)

    for word in y:
      preprocess_word = tokenize(word, target_alphabet)
      preprocess_y.append(preprocess_word)

    pad_list_x = []
    pad_list_y = []

    for vectorized_word in preprocess_x:
      pad_list_x.append(pad(vectorized_word))

    for vectorized_word in preprocess_y:
      pad_list_y.append(pad(vectorized_word))

    preprocess_x = pad_list_x
    preprocess_y = pad_list_y

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    # preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y

preproc_german_sentences, preproc_english_sentences = preprocess(english_sentences, french_sentences, input_alphabet, output_alphabet)


In [174]:
import tensorflow as tf

preproc_english_sentences = preproc_english_sentences[:400]
X_train = np.array(preproc_english_sentences)
preproc_english_sentences = preproc_english_sentences[:400]
y_train = np.array(preproc_english_sentences)
# y_train = y_train.reshape(*y_train.shape, 1)
X_dev = tf.constant([3.0, 1.0, 2.0])
y_dev = tf.constant([0.0, 2.0, 1.0])

In [178]:
BATCH_SIZE, HIDDEN_SIZE = 32, 16
NUM_ITERATIONS = 10
NUM_EPOCHS_PER_ITERATION = 100

model = Sequential()
model.add(
    GRU(
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(42, 1),
        unroll=True
    )
)
model.add(Dense(42))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

for iteration in range(NUM_ITERATIONS):

    print("=" * 50)
    if (iteration % 10 == 0):
      print("Iteration #: %d" % (iteration))
    model.fit(X_train[:, :, np.newaxis], y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

Iteration #: 0
Epoch 1/100
13/13 [==============================] - 17s 7ms/step - loss: 277.1926
Epoch 2/100
13/13 [==============================] - 0s 7ms/step - loss: 271.0471
Epoch 3/100
13/13 [==============================] - 0s 7ms/step - loss: 253.6848
Epoch 4/100
13/13 [==============================] - 0s 7ms/step - loss: 212.1082
Epoch 5/100
13/13 [==============================] - 0s 7ms/step - loss: 200.2190
Epoch 6/100
13/13 [==============================] - 0s 7ms/step - loss: 203.2032
Epoch 7/100
13/13 [==============================] - 0s 7ms/step - loss: 198.0027
Epoch 8/100
13/13 [==============================] - 0s 8ms/step - loss: 188.3957
Epoch 9/100
13/13 [==============================] - 0s 8ms/step - loss: 195.0553
Epoch 10/100
13/13 [==============================] - 0s 8ms/step - loss: 197.4592
Epoch 11/100
13/13 [==============================] - 0s 8ms/step - loss: 187.4231
Epoch 12/100
13/13 [==============================] - 0s 9ms/step - loss: 194.47

In [179]:
pred = model.predict(X_train[:, :, np.newaxis], verbose=0)[0]

In [180]:
pred

array([3.82555693e-01, 3.94929536e-02, 2.35430524e-01, 1.03304967e-01,
       1.77975520e-01, 4.63540666e-02, 1.33609492e-02, 1.42012350e-03,
       1.03003775e-04, 2.18653508e-06, 3.06819214e-09, 8.11677030e-15,
       8.36036253e-19, 1.73900997e-25, 4.17283849e-36, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00], dtype=float32)

In [157]:
np.shape(X_train)

(400, 42)

In [159]:
np.shape(X_train[:, :, np.newaxis])

(400, 42, 1)

In [154]:
y_train

array([[ 0,  0,  1, ...,  0,  0,  0],
       [ 0,  0,  1, ...,  0,  0,  0],
       [ 0,  0,  1, ...,  0,  0,  0],
       ...,
       [ 8, 19, 15, ...,  0,  0,  0],
       [ 8, 19, 15, ...,  0,  0,  0],
       [15,  4, 12, ...,  0,  0,  0]])